In [2]:
words = open('names.txt', 'r').read().splitlines()

In [3]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [4]:
len(words)

32033

In [5]:
min(len(w) for w in words)

2

In [6]:
max(len(w) for w in words)

15

In [7]:
# we will keep a counter b for each bigram
b = {}

for w in words:
    # we will use <S> = start and <E> = end for each word
    chs = ['<S>'] + list(w) + ['E']
    for ch1, ch2 in zip(chs, chs[1:]):
        # construct bigram of the two adjacent chars
        bigram = (ch1, ch2)
        # increment counter
        b[bigram] = b.get(bigram, 0) + 1

In [ ]:
b

In [ ]:
# sort the dict by value in decreasing order
sorted(b.items(), key = lambda pair: pair[1], reverse=True)

In [10]:
import torch

In [ ]:
### represent this bigram counter as a 2D array (tensor)
### we will use '.' as the start/end char

# we will use int32 instead of float32 since all our values are ints
# size is 27 (26 chars + 1 start/end char)
N = torch.zeros((27, 27), dtype=torch.int32)

# get all possible values for a character in the names list
chars = sorted(list(set(''.join(words))))

# create a LUT char -> int
stoi = {c: i+1 for i, c in enumerate(chars)}
# add the start/end char
stoi['.'] = 0

# reverse lookup table to get int -> char LUT
# offset all the i's by +1 since we want '.' to be at index 0
itos = {c: i for i, c in stoi.items()}
itos

In [12]:
### populate the 2D array counter
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        # find indices with LUT
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
### visualize the bigram counter

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha='center', va='bottom', color='gray')
        plt.text(j, i, N[i, j].item(), ha='center', va='top', color='gray')
plt.axis('off')

In [14]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [19]:
# we can turn a distribution into a probability distribution by dividing by the sum
p = N[0].float()
p /= p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [51]:
# we can use a generator to pick samples from a probability distribution
# use the same seed for the generator produces deterministic results
# g = torch.Generator().manual_seed(2147483647)
# p = torch.rand(3, generator=g)
# p /= p.sum()
# p

In [31]:
# sample an index
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'j'

In [ ]:
g = torch.Generator().manual_seed(2147483647)

### GENERATE NAMES
for i in range(10):
    ### DEFINE SAMPLING LOOP
    # start with ix = 0, since 0 -> '.' which is the start/end char
    ix = 0
    # output
    out = []
    while True:
        # get the probability distrbution for the char after c = itos[ix]
        p = N[ix].float()
        # normalize the probability distribution
        p = p / p.sum()
        # sample for next char index
        ix = torch.multinomial(P, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    
    print(''.join(out))

# OPTIMIZATIONS
#     - normalize each row of N before the loop to avoid performing the same division multiple times
#     - use in-place division with the /= operator to avoid allocating memory for the additional p array

In [90]:
### NORMALIZE PROBABILITY MATRIX
P = N.float()
# we want to sum along the rows; the columns should be collapsed into a single column
# row = 0, column = 1 => dim = 1
# we also need keepdim=True, otherwise the dim=1 is flattened out, and we get a (27) tensor or a row vector
P /= P.sum(1, keepdim=True)

# more on broadcasting: https://pytorch.org/docs/stable/notes/broadcasting.html


# we are trying to divide a (27, 27) tensor by a (27, 1) tensor

# 27, 27
# 27,  1

# in this case, torch takes the tensor where the dim is 1,
# and copies it horizontally 27 times 

# example:

# a       a a a a  ...  a a a a
# b       b b b b  ...  b b b b
# .                 .
# .   ->            .
# .                 .
# y       y y y y  ...  y y y y
# z       z z z z  ...  z z z z

In [91]:
g = torch.Generator().manual_seed(2147483647)

### GENERATE NAMES
for i in range(10):
    ### DEFINE SAMPLING LOOP
    # start with ix = 0, since 0 -> '.' which is the start/end char
    ix = 0
    # output
    out = []
    while True:
        # get the normalized probability distrbution for the char after c = itos[ix]
        p = P[ix]
        # sample for next char index
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    
    print(''.join(out))

junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juee.
ksahnaauranilevias.


In [112]:
### EVALUATING THE LOSS

# in maximum likelihood estimation, we typically use LIKELIHOOD
# likelihood is product of the all the probabilities

# since our probabilities are all less than 1, the product will be a very small number
# instead, we will take the NEGATIVE LOG (NEGATIVE LOG LIKELIHOOD or NLL)
# useful since log(a*b*c) = log(a) + log(b) + log(c); we can move between product and sum
# we make it negative just to stay consistent with the semantics of a loss function, where low == good

log_likelihood = 0.0
n = 0

for w in ["fdafsjq"]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        # find indices with LUT
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        # find probability of this bigram
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        # print(f'{ch1}{ch2}: {logprob:.4f}')
        
nll = -log_likelihood

# the AVERAGE NLL is important, since each bigram will increase the loss
avg_nll = nll / n

print(f'avg_nll: {avg_nll}')

# GOAL: maximize likelihood
# -> maximize log likelihood (since log is increasing)
# -> minimize negative log likelihood
# -> minimize average log likelihood

avg_nll: 5.157317638397217


In [111]:
### PROBLEM!!!
# if an input has a bigram in it that is not in the training set,
# i.e. its count is 0,
# we get log(0) = -inf which is very bad

# to fix this, we can just add 1 to all the counts
# this doesn't really change the probability distributions, but avoids the inf case
P = (N + 1).float()
P /= P.sum(1, keepdim=True)